# Machine Leanrning Final Project ---- Music AI

This is the notebook we used to perform classification on the VGG data

In [ ]:
import json
import os
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
os.chdir("drive/My Drive/ML final project/openmic-2018")
!ls "/content/drive/My Drive/ML final project/openmic-2018"

acknowledgement.md  license-cc-by.txt			   openmic-2018.npz
audio		    openmic-2018-aggregated-labels.csv	   partitions
checksums	    openmic-2018-individual-responses.csv  vggish
class-map.json	    openmic-2018-metadata.csv


In [ ]:
labels = pd.read_csv("openmic-2018-aggregated-labels.csv")
print(labels)
print(np.shape(labels))
print(type(labels))

          sample_key instrument  relevance  num_responses
0        000046_3840   clarinet    0.17105              3
1        000046_3840      flute    0.00000              3
2        000046_3840    trumpet    0.00000              3
3      000135_483840  saxophone    0.14705              3
4      000135_483840      voice    1.00000              3
...              ...        ...        ...            ...
41529  155307_211200  saxophone    1.00000              3
41530  155310_372480  saxophone    1.00000              3
41531  155311_453120  saxophone    1.00000              3
41532  155311_453120    trumpet    0.82605              3
41533  155311_453120   trombone    1.00000              3

[41534 rows x 4 columns]
(41534, 4)
<class 'pandas.core.frame.DataFrame'>


In [ ]:
metadata = pd.read_csv("openmic-2018-metadata.csv")
print(metadata)
print(np.shape(metadata))
#print(type(metadata))

       track_id  album_id                                  album_title  \
0            46       4.0                                        Niris   
1           135      58.0                                          mp3   
2           139      60.0                        Every Man For Himself   
3           141      60.0                        Every Man For Himself   
4           144      64.0                                   Amoebiasis   
...         ...       ...                                          ...   
19995    155294   22935.0                                       Return   
19996    155295   22935.0                                       Return   
19997    155307   22937.0  Live at WFMU with Scott Williams, 3/27/2017   
19998    155310   22937.0  Live at WFMU with Scott Williams, 3/27/2017   
19999    155311   22937.0  Live at WFMU with Scott Williams, 3/27/2017   

                                               album_url  artist_id  \
0      http://freemusicarchive.org/music

In [ ]:
# responses = pd.read_csv("openmic-2018-individual-responses.csv")
# print(responses)
# print(np.shape(responses))
# print(type(responses))

## 1] Clasification based purely on metadata




NOTE: The loading of the data (the following block only) was adapted from https://github.com/cosmir/openmic-2018/blob/master/examples/modeling-baseline.ipynb, but the methods following it were implemented ourselves by using sklearn. Using the link given for the following block helped us understand our dataset a lot more.

In [ ]:
# Quick note on the terms:
# .json: VGGish features as JSON objects, in a directory format like 'vggish/{0:3}/{0}.json'.format(sample_key)
# .ogg: 10 second snippets of audio, in a directory format like 'audio/{0:3}/{0}.ogg'.format(sample_key)
# .npz: A Python-friendly NPZ file of features and labels, 'openmic-2018.npz'
# VGGish: 


# Now we need to read in .json file for each track and transform them into
# spectrograms for latter input into the neural network
openmic = np.load('openmic-2018.npz', allow_pickle=True)
# Use allow_pickle = True so later the sample_key could be properly loaded


# X: (20000 by 10 by 128) array of VGGish features
#     First index (0..19999) corresponds to the sample key
#     Second index (0..9) corresponds to the time within the clip (each time slice is 960 ms long)
#     Third index (0..127) corresponds to the VGGish features at each point in the 10sec clip
#     Example X[40, 8] is the 128-dimensional feature vector for the 9th time slice in the 41st example

X = openmic['X']

# Y_true: 20000 20 array of true* label probabilities
#      First index corresponds to sample key, as above
#      Second index corresponds to the label class (accordion, ..., voice)
#      Example: Y[40, 4] indicates the confidence that example #41 contains the 5th instrument

Y_true = openmic['Y_true']


# Y_mask: 20000 * 20 binary mask values
#    First index corresponds to sample key
#    Second index corresponds to the label class
#    Example: Y[40, 4] indicates whether or not we have observations for the 5th instrument for example #41

Y_mask = openmic['Y_mask']

# sample_key: 20000 array of sample key strings
#     Example: sample_key[40] is the sample key for example #4

sample_key = openmic['sample_key']

# Import the class map for convenience
f = open('class-map.json')
class_map = json.load(f)
f.close()

In [ ]:
### only the code in the block above was adapted from the link given

print("Data: \n", list(openmic.keys()))

print("\n\nX shape: \n", np.shape(X))

print("\n\nExample of X values (X[0]): \n", X[0])

print("\n\nY shape (labels): \n", np.shape(Y_true))

print("\n\nExample of y values (Y_true[0]): \n", Y_true[0])

print("\n\nShape of sample key identifier array: \n", np.shape(sample_key))

print("\n\nFirst sample key, identifies song because the audio files are named by the sample key: \n", sample_key[0])

print("\n\nWe are doing multiclass classification, and these are the instruments that correspond to each label: \n", class_map)

Data: 
 ['X', 'Y_true', 'Y_mask', 'sample_key']


X shape: 
 (20000, 10, 128)


Example of X values (X[0]): 
 [[173  16 162 ...  42  37 255]
 [175  13 171 ...  55  82 255]
 [176  14 174 ... 153  59 255]
 ...
 [195  39 210 ...   0   0 255]
 [188  21 176 ...   0  96 255]
 [193  23 178 ...   0 200 255]]


Y shape (labels): 
 (20000, 20)


Example of y values (Y_true[0]): 
 [0.5     0.5     0.5     0.5     0.17105 0.5     0.5     0.      0.5
 0.5     0.5     0.5     0.5     0.5     0.5     0.5     0.      0.5
 0.5     0.5    ]


Shape of sample key identifier array: 
 (20000,)


First sample key, identifies song because the audio files are named by the sample key: 
 000046_3840


We are doing multiclass classification, and these are the instruments that correspond to each label: 
 {'accordion': 0, 'banjo': 1, 'bass': 2, 'cello': 3, 'clarinet': 4, 'cymbals': 5, 'drums': 6, 'flute': 7, 'guitar': 8, 'mallet_percussion': 9, 'mandolin': 10, 'organ': 11, 'piano': 12, 'saxophone': 13, 'synthesize

In [ ]:
# Currently, X[i] gives the VGGish features for the ith song.
# However, it is a 2d array (10 x 128), where the X[i][k] represents the kth second of the 10 second video; each second has 128 VGGish features

# Here, we flatten the 2nd and 3rd dimensions into 1 so that we can use all the VGGish values as features for the ith sample/song in sklearn easily:
flatDim = X.shape[1] * X.shape[2]
numSamples = X.shape[0]
X_flat = X.reshape(numSamples, flatDim)

print(X_flat.shape)

(20000, 1280)


In [ ]:
# We want to do multiclass classification in order to find the most prevalent instrument in a given song (this is because sklearn does not support multilabel classification for many of its
# classifiers, so we are doing multiclass classification, which is supported by most classifiers. Because of this, we can compare many classifiers). 
# So, since we are given an array of probabilities that a given song has a given instrument in it, we can take the max of the label array for each 
# song and use that instrument as its label. Note that if there is no data on an instrument in a song, the probability is .5 (equally likely either way),
# so we use the mask to zero out the probability of non-observed instruments

# For example:
y1 = Y_true[1] * Y_mask[1]
print(y1)

# The song has 2 observed instrument values, and the probability of the last one (voice) is the highest. So, when we take the max:
y1_label = np.argmax(y1)
print("\n\nlabel for song 2: ", y1_label)
# we get 19, which indicates that the 19th "instrument" (voice, in this case) is the most likely to be in the song.


[0.      0.      0.      0.      0.      0.      0.      0.      0.
 0.      0.      0.      0.      0.14705 0.      0.      0.      0.
 0.      1.     ]


label for song 2:  19


In [ ]:
# We will now apply this label conversion to the whole dataset:
y = Y_true * Y_mask
y = np.argmax(y, axis=1)

# y[i] now contains the single label for X[i] that represents the most common/likely/prevalent instrument used in that song

print(np.unique(y))
# And we see that each instrument is the most prevalent in at least one song in the dataset

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [ ]:
print(X_flat.shape)
print(y.shape)
# our data is now ready to be used in sklearn

(20000, 1280)
(20000,)


In [ ]:
# after our presentation, we tried reducing the dimensionality of our date using PCA, which greatly helped. See the Clustering notebook (also in our codebase) for how we
# came up with the number 182

from sklearn.decomposition import PCA
pca2 = PCA(n_components=182)
X_reduced = pca2.fit_transform(X_flat)

In [ ]:
# generate train/test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.1)


print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16000, 182)
(4000, 182)
(16000,)
(4000,)


In [ ]:
# normalize the data

from sklearn import preprocessing

scaler_train = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler_train.transform(X_train)

scaler_test = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler_test.transform(X_test)

In [ ]:
# First model: logistic regression

from sklearn.linear_model import LogisticRegression

# multinomial multiclass option
clfLogisticMultinomial = LogisticRegression(multi_class="multinomial", max_iter=700, solver="newton-cg").fit(X_train_scaled, y_train)

# one-vs-all multiclass option
clfLogisticOVR = LogisticRegression(multi_class="ovr", max_iter=700, solver="newton-cg").fit(X_train_scaled, y_train)

print("Multinomial logistic regression score: ", clfLogisticMultinomial.score(X_test_scaled, y_test))
print("One-vs-all logistic regression score: ", clfLogisticOVR.score(X_test_scaled, y_test))

Multinomial logistic regression score:  0.605
One-vs-all logistic regression score:  0.60925


In [ ]:
### logistic regression gives a score of ~.63 with PCA, .5 without

In [ ]:
from sklearn.svm import LinearSVC

clfSVM_ovr = LinearSVC(multi_class="ovr").fit(X_train_scaled, y_train)

#clfSVM_cs = LinearSVC(multi_class="crammer_singer").fit(X_train_scaled, y_train)


print("One-vs-all SVM score: ", clfSVM_ovr.score(X_test_scaled, y_test))
#print("crammer singer SVM score: ", clfSVM_cs.score(X_test_scaled, y_test))

One-vs-all SVM score:  0.5975


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clfForest = RandomForestClassifier(n_estimators=800).fit(X_train_scaled, y_train)

print("Random forest score: ", clfForest.score(X_test_scaled, y_test))

Random forest score:  0.567


In [ ]:
### random forests give a score of ~.57

In [ ]:
# class distributions
tot = len(y)
for q in np.unique(y):
  print("class", q, ": ", (y == q).sum() / tot * 100, "%")

class 0 :  18.759999999999998 %
class 1 :  3.7900000000000005 %
class 2 :  3.795 %
class 3 :  4.555 %
class 4 :  2.22 %
class 5 :  5.62 %
class 6 :  4.64 %
class 7 :  4.125 %
class 8 :  5.57 %
class 9 :  4.17 %
class 10 :  3.345 %
class 11 :  4.075 %
class 12 :  5.6899999999999995 %
class 13 :  5.045 %
class 14 :  5.35 %
class 15 :  3.17 %
class 16 :  3.16 %
class 17 :  2.9899999999999998 %
class 18 :  5.045 %
class 19 :  4.885 %
